In [25]:
import os
import numpy as np
import random
import torch
import time
from train_dqn import fill_memory, train_dqn
%matplotlib inline

%load_ext autoreload
%autoreload 2

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
cuda


In [26]:
seed = 1234
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

In [27]:
from envs.GraphEnv.impnode import ImpnodeEnv
import DQN_agent_modular

In [28]:
# variables for training the agent

num_train_eps = 1000000 #10000 number training episodes to run
num_mem_fill_eps = 1000 #1000 number of episodes to run to initialize the memory
discount = 0.99 # gamma used for computing return
batch_size = 64 # number of transitions to sample from replay buffer for each learn step
mem_capacity = 500000  # 500000 size of the memory buffer
update_frequency = 1000 #1000 number of interactions after which the target buffer is updated
eps_max = 1.0 # initial epsilon value
eps_min = 0.05 # final epsilon value
eps_step = 10000 #1000 amount by which epsilon is decayed at each episode
lr = 0.0001 # learning rate for the network

gnn_depth=4
state_size=5
hidden_size1=16
hidden_size2=64
action_size=1
n_step=5

ba_nodes = (15, 25)
ba_edges=4
anc = 'dw_nd'
max_removed_nodes = 4
seed = None


In [29]:
results_base_path_train = "results/1".format(
                        device,ba_nodes,ba_edges,max_removed_nodes,num_train_eps,num_mem_fill_eps,discount,batch_size,mem_capacity, update_frequency,eps_max,eps_min, eps_step,lr)

if not os.path.exists(results_base_path_train):
    os.makedirs(results_base_path_train)
else:
    timestamp = time.strftime("%Y%m%d%H%M%S")
    new_directory = f"{results_base_path_train}_{timestamp}"
    os.makedirs(new_directory)
    results_base_path_train = new_directory
    print(f"Directory already exists. Created a new one: {new_directory}")



Directory already exists. Created a new one: results/1_20240321093922


In [30]:
env_train = ImpnodeEnv(anc=anc, 
                       ba_nodes=ba_nodes, 
                       ba_edges=ba_edges, 
                       max_removed_nodes=max_removed_nodes, 
                       seed=seed, 
                       render_option=False, 
                       train_mode=True)

In [31]:
# create the dqn_agent
dqn_agent_train = DQN_agent_modular.DQNAgent(device,
                                             gnn_depth=gnn_depth,
                                             state_size=state_size,
                                             hidden_size1=hidden_size1,
                                             hidden_size2=hidden_size2,
                                             action_size=action_size,
                                             discount=discount,
                                             eps_max=eps_max,
                                             eps_min=eps_min,
                                             eps_step=eps_step,
                                             memory_capacity=mem_capacity,
                                             lr=lr,
                                             train_mode=True)



In [32]:
fill_memory(env_train, dqn_agent_train, num_mem_fill_eps, n_step)
train_dqn(env_train, dqn_agent_train, results_base_path_train, num_train_eps, n_step, batch_size, update_frequency)

KeyboardInterrupt: 

In [ ]:
print('done')